In [1]:
from __future__ import print_function
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [59]:
#Defining some parameters (these parameters vary for different images and are to be adjusted for every image included in the test dataset)
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.25

In [60]:
#Defining a function to display images in RGB format
def display(img, cmap = None):
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap = cmap)

In [61]:
#Function to transform the test image to match the template
def align_images(im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  # Match features.
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1,descriptors2)
    matches = sorted(matches, key = lambda x:x.distance)
  # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)
  # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imwrite("matches.jpg", imMatches)
  # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32) 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
  # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    return im1Reg, h

In [62]:
if __name__ == '__main__':
    
  # Read reference image
    refFilename = "sweatpatch_template.JPG"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
    
  # Read image to be aligned
    imFilename = "test_image.jpg"
    print("Reading image to align : ", imFilename);
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
    print("Aligning images ...")
 # The estimated homography will be stored in h.
 # Registered image will be resotred in imReg.
    imReg, h = align_images(im, imReference)
    outFilename = "aligned.JPG"
 # Write aligned image to disk.
    print("Saving aligned image : ", outFilename);
    cv2.imwrite(outFilename, imReg)   
  # Print estimated homography
    print("Estimated homography : \n",  h)
   

Reading reference image :  sweatpatch_template.JPG
Reading image to align :  test_image.jpg
Aligning images ...
Saving aligned image :  aligned.JPG
Estimated homography : 
 [[ 6.54086222e-02  1.01733173e+00 -1.11167159e+03]
 [-9.67315438e-01  1.46043303e-02  2.16873249e+03]
 [ 5.95193284e-05  4.52074163e-05  1.00000000e+00]]
